<a href="https://colab.research.google.com/github/Witcape/AI_Practice/blob/main/trt_conversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import time
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.python.compiler.tensorrt import trt_convert as trt
from tensorflow.python.saved_model import tag_constants
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

In [2]:
model = ResNet50(weights = 'imagenet')
model.export('resnet50_saved_model')

102967424/102967424 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Saved artifact at 'resnet50_saved_model'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1000), dtype=tf.float32, name=None)
Captures:
  133263665742672: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133263665747280: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133263665747088: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133263665746896: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133263665742096: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133263665746320: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133263665750160: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133263665751312: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133263665752272: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133263665751696: TensorS

In [6]:
def predict_and_benchmark_throughput_from_saved(batched_input, infer, N_warmup_run=50, N_run=500, model='resnet_v2'):
    elapsed_time = []
    all_preds = []
    batch_size = batched_input.shape[0]

    label_key = None

    if model == 'resnet_v2':
        label_key = 'probs'
    elif model in ['vgg19', 'inception_v3']:
        label_key = 'predictions'
    else:
        raise ValueError("Unsupported model name: %s" % model)

    for i in range(N_warmup_run):
        labeling = infer(batched_input)
#         preds = labeling[label_key].numpy()


    for i in range(N_run):
        start_time = time.time()

        # This call to infer is asynchronous, therefore, for the sake of benchmarking...
        labeling = infer(batched_input)

        # ...we demand a return value to force the execution to wait, before ending the timer
#         preds = labeling[label_key].numpy()

        end_time = time.time()
        elapsed_time = np.append(elapsed_time, end_time - start_time)

#         all_preds.append(preds)

        if i % 50 == 0:
            print('Steps {}-{} average: {:4.1f}ms'.format(i, i+50, (elapsed_time[-50:].mean()) * 1000))

    throughput = N_run * batch_size / elapsed_time.sum()
    print(f'Throughput: {throughput:.0f} images/s')
    return all_preds, throughput

In [7]:
input_saved_model_dir = '/content/resnet50_saved_model/'
print(f'Loading saved model {input_saved_model_dir}')
saved_model_loaded = tf.saved_model.load(input_saved_model_dir, tags = [tag_constants.SERVING])
infer = saved_model_loaded.signatures['serving_default']

BATCH_SIZE = 32
dummy_input_batch = np.zeros((BATCH_SIZE, 224, 224, 3), dtype = np.float32)
dummy_input_batch = tf.constant(dummy_input_batch)

Loading saved model /content/resnet50_saved_model/


In [8]:
predict_and_benchmark_throughput_from_saved(dummy_input_batch, infer, N_warmup_run = 5,
                                            N_run = 500, model = 'resnet_v2')

Steps 0-50 average: 107.1ms
Steps 50-100 average: 107.3ms
Steps 100-150 average: 110.9ms
Steps 150-200 average: 108.4ms
Steps 200-250 average: 108.9ms
Steps 250-300 average: 109.4ms
Steps 300-350 average: 109.8ms
Steps 350-400 average: 110.7ms
Steps 400-450 average: 110.9ms
Steps 450-500 average: 111.3ms
Throughput: 291 images/s


([], 291.0732432093265)

In [9]:
conversion_params = trt.DEFAULT_TRT_CONVERSION_PARAMS._replace(precision_mode = trt.TrtPrecisionMode.FP16,
                                                               max_workspace_size_bytes = 8000000000)

converter = trt.TrtGraphConverterV2(input_saved_model_dir = '/content/resnet50_saved_model'
                                    , conversion_params = conversion_params)

converter.convert()
converter.save(output_saved_model_dir = '/content/resnet50_saved_model_2')
print('Done Converting to TF-TRT FP16')

ERROR:tensorflow:Tensorflow needs to be built with TensorRT support enabled to allow TF-TRT to operate.


RuntimeError: Tensorflow has not been built with TensorRT support.